In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from IPython.display import display, clear_output
from ipywidgets import *

from magpie import magpie

In [2]:
%matplotlib inline

Lx = 1.1; Ly = 1.1; Lz = 5e-3
ldim = [Lx, Ly, Lz]  # -- plate dimensions [x, y, z] in metres
E = 9.0e+9           # -- Young's mod [Pa]
rho = 8765           # -- density [kg/m^3]
nu = 0.3             # -- poisson's ratio
Nmodes = 4           # -- number of modes to compute
h = np.sqrt(Lx * Ly) * 0.05   # -- grid spacing
BCs = np.ones((4, 2)) * 1e15 # -- elastic constants around the edges

chladni_plot = Output();


def plot_mode_shapes(K0y, Kx0, KLy, KxL, R0y, Rx0, RLy, RxL):
    """
    """
    
    BCs = np.array([[Kx0,Rx0],[K0y,R0y],[KxL,RxL],[KLy,RLy]], dtype=np.float64)
    m = 2

    with chladni_plot:
        
        Q, Om, N, biharm = magpie(rho, E, nu, ldim, h, BCs, Nmodes)
        for m in range(Nmodes):
            mode_shapes = [np.real(np.reshape(Q[:, m], [N['x'], N['y']])) for m in range(Nmodes)]

        clear_output(wait=True)
        
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)

        Z = abs(mode_shapes[m])
        chladni = plt.pcolormesh(Z.T, cmap='copper_r', shading='gouraud')
        ax.set_axis_off()
        cmax = np.max(abs(mode_shapes[m])) * 0.25
        chladni.set_clim(0.00, cmax)
        

        plt.plot()        
        plt.show()        

In [53]:
labels = ['K0y', 'Kx0', 'KLy', 'KxL', 'R0y', 'Rx0', 'RLy', 'RxL']

sliders =  {label:FloatLogSlider(
    value=0.0, base=10, min=0, max=15,
    description=label,
    continuous_update=True,
    orientation='vertical' if 'x' in label else 'horizontal',
    readout_format='.1e',
    layout=Layout(flex='1 1 1', width='auto')
) for label in labels}


sliders['Rx0'] = FloatSlider(value=0.05, min=113.99043015, max=113.990430165, step=0.0000000001,readout=True ,orientation='vertical')

interactive_output(plot_mode_shapes, sliders)


Output()

In [4]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center'
)

form_items = [
    Box([Label(value='Dimensions'), 
         BoundedFloatText(min=0.2, max=2.0, description='X [m]'),
         BoundedFloatText(min=0.2, max=2.0, description='Y [m]'),
         BoundedFloatText(min=0.001, max=0.01, description='thickness [m]')], layout=form_item_layout),
    Box([Label(value='Material'), 
         Dropdown(options=["steel", "gold", "silver", "copper", "aluminium"])], layout=form_item_layout),
    Box([Label(value=''), 
         BoundedFloatText(min=0.2, max=2.0, description='Young\'s',disabled=True,),
         BoundedFloatText(min=0.2, max=2.0, description='Density',disabled=True,),
         BoundedFloatText(min=0.001, max=0.01, description='Poisson\'s',disabled=True,)], layout=form_item_layout),
    Box([Label(value='Resolution'), FloatSlider(value=0.05, min=0.005, max=0.05, step=0.001, 
                                                readout=True, readout_format='.3f',)], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='auto',
))



In [54]:

plot_mode_shapes(*[s.value for s in sliders.values()])

display(
    VBox([form,
          VBox([sliders['KLy'],sliders['RLy']]),
          HBox([sliders['Kx0'],sliders['Rx0'],chladni_plot, sliders['KxL'],sliders['RxL']]),
          VBox([sliders['K0y'],sliders['R0y']])])
)